In [1]:
!pip install langchain langchain-community langchain_openai tiktoken faiss-cpu chromadb wikipedia openai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_community.retrievers import WikipediaRetriever

In [3]:
# Initialize the retriever(optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [4]:
# Define query
query = "The geopolitical history of India and Pakistan from the perspective of a chinese"

# Get wikipedia documents
docs = retriever.invoke(query)

In [6]:
for idx, val in enumerate(docs):
  print(f"____Result:{idx + 1}_____")
  print(f"Content: \n{val.page_content}\n")

____Result:1_____
Content: 
The India–Pakistan war of 1965, also known as  the second Kashmir war, was an armed conflict between Pakistan and India that took place from August 1965 to September 1965. The conflict began following Pakistan's unsuccessful Operation Gibraltar, which was designed to infiltrate forces into Jammu and Kashmir to precipitate an insurgency against Indian rule. The seventeen day war caused thousands of casualties on both sides and witnessed the largest engagement of armoured vehicles and the largest tank battle since World War II. Hostilities between the two countries ended after a ceasefire was declared through UNSC Resolution 211 following a diplomatic intervention by the Soviet Union and the United States, and the subsequent issuance of the Tashkent Declaration. Much of the war was fought by the countries' land forces in Kashmir and along the border between India and Pakistan. This war saw the largest amassing of troops in Kashmir since the Partition of India 

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [8]:
import os

In [10]:
os.environ["OPENAI_API_KEY"] = ""

In [11]:
# Source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models.")
]

In [12]:
# Initialize the embedding model
embedding_model = OpenAIEmbeddings()

# Create chroma vector store in memory
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name = "my_sample_collection"
)

In [13]:
# Convert Vector store into a retriever
retriever = vector_store.as_retriever(search_kwargs={"k":2})

In [15]:
query = "What is Chroma used for?"
result = retriever.invoke(query)

In [16]:
for idx, val in enumerate(result):
  print(f"____Result:{idx + 1}_____")
  print(f"Content: \n{val.page_content}\n")

____Result:1_____
Content: 
Chroma is a vector database optimized for LLM-based search.

____Result:2_____
Content: 
LangChain helps developers build LLM applications easily.



In [17]:
# MMR: Maximum Marginal Relevance
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more.")
]

In [18]:
from langchain_community.vectorstores import FAISS

In [20]:
embedding2 = OpenAIEmbeddings()

# create faiss vector store
vector_s = FAISS.from_documents(
    documents=docs,
    embedding=embedding2
)

In [28]:
faiss_retriever = vector_s.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3, "lambda_mult":1}
)

In [29]:
query2 = "What is Langchain?"
results = faiss_retriever.invoke(query2)

In [32]:
for idx, val in enumerate(results):
  print(f"____Result:{idx + 1}_____")
  print(f"Content: \n{val.page_content}\n")

____Result:1_____
Content: 
LangChain is used to build LLM based applications.

____Result:2_____
Content: 
Embeddings are vector representations of text.

____Result:3_____
Content: 
MMR helps you get diverse results when doing similarity search.

____Result:4_____
Content: 
Chroma is used to store and search document embeddings.



In [36]:
!pip install -U langchain langchain-community langchain-openai

In [37]:
!pip install langchain-classic

# Multiquery Retrieval

In [38]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_classic.retrievers import MultiQueryRetriever

In [40]:
# Relevant health & wellness documents
all_docs = [
    Document(
        page_content="Regular walking boosts heart health and can reduce symptoms of depression.",
        metadata={"source": "H1"}
    ),
    Document(
        page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.",
        metadata={"source": "H2"}
    ),
    Document(
        page_content="Deep sleep is crucial for cellular repair and emotional regulation.",
        metadata={"source": "H3"}
    ),
    Document(
        page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.",
        metadata={"source": "H4"}
    ),
    Document(
        page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.",
        metadata={"source": "H5"}
    ),
    Document(
        page_content="The solar energy system in modern homes helps balance electricity demand.",
        metadata={"source": "I1"}
    ),
    Document(
        page_content="Python balances readability with power, making it a popular system design language.",
        metadata={"source": "I2"}
    ),
    Document(
        page_content="Photosynthesis enables plants to produce energy by converting sunlight.",
        metadata={"source": "I3"}
    ),
    Document(
        page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.",
        metadata={"source": "I4"}
    ),
    Document(
        page_content="Black holes bend spacetime and store immense gravitational energy.",
        metadata={"source": "I5"}
    )
]


In [42]:
# Initialize the embedding
embedding3 = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=all_docs,
                                   embedding=embedding3,
                                   )

In [43]:
similarity_retriever = vectorstore.as_retriever(search_type="similarity",
                                                search_kwargs={"k": 5})


In [44]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k":5}),
    llm=ChatOpenAI(model="gpt-3.5-turbo")
)

In [45]:
# Query
query3 = "How to improve energy levels and maintain balance?"

In [46]:
similarity_results = similarity_retriever.invoke(query3)
multiquery_results = multiquery_retriever.invoke(query3)

In [47]:
for i,doc in enumerate(similarity_results):
  print(f"____Result:{i + 1}_____")
  print(f"Content: \n{doc.page_content}\n")

print("*"*150)

for i,doc in enumerate(multiquery_results):
  print(f"____Result:{i + 1}_____")
  print(f"Content: \n{doc.page_content}\n")


____Result:1_____
Content: 
Drinking sufficient water throughout the day helps maintain metabolism and energy.

____Result:2_____
Content: 
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

____Result:3_____
Content: 
Regular walking boosts heart health and can reduce symptoms of depression.

____Result:4_____
Content: 
Deep sleep is crucial for cellular repair and emotional regulation.

____Result:5_____
Content: 
The solar energy system in modern homes helps balance electricity demand.

******************************************************************************************************************************************************
____Result:1_____
Content: 
Drinking sufficient water throughout the day helps maintain metabolism and energy.

____Result:2_____
Content: 
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

____Result:3_____
Content: 
Deep sleep is crucial for cellular repair and emotional regulation.

____Re

In [49]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

In [50]:
docs = [
    Document(
        page_content="""
The Grand Canyon is one of the most visited natural wonders in the world.
Photosynthesis is the process by which green plants convert sunlight into energy.
Millions of tourists travel to see it every year. The rocks date back millions of years.""",
        metadata={"source": "Doc1"}
    ),

    Document(
        page_content="""
In medieval Europe, castles were built primarily for defense.
Chlorophyll in plant cells captures sunlight during photosynthesis.
Knights wore armor made of metal. Siege weapons were often used to breach castle walls.""",
        metadata={"source": "Doc2"}
    ),

    Document(
        page_content="""
Basketball was invented by Dr. James Naismith in the late 19th century.
It was originally played with a soccer ball and peach baskets.
NBA is now a global league.""",
        metadata={"source": "Doc3"}
    ),

    Document(
        page_content="""The history of cinema began in the late 1800s.
Silent films were the earliest form.Thomas Edison was among the pioneers.
Photosynthesis does not occur in animal cells. Modern film making involves complex CGI and sound design.""",
        metadata={"source": "Doc4"}
    ),
]


In [51]:
embedding4 = OpenAIEmbeddings()
vecstor = FAISS.from_documents(documents=docs, embedding=embedding4)

In [52]:
llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

In [53]:
base_retriever = vecstor.as_retriever(search_kwargs={"k":5})

In [54]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [56]:
query = "what is photosynthesis?"
compressed_res = compression_retriever.invoke(query)

In [57]:
for i, doc in enumerate(compressed_res):
  print(f"____Result:{i + 1}_____")
  print(f"Content: \n{doc.page_content}\n")

____Result:1_____
Content: 
Photosynthesis is the process by which green plants convert sunlight into energy.

____Result:2_____
Content: 
Chlorophyll in plant cells captures sunlight during photosynthesis.

